# Forecasting with ARIMA

### Imports

In [15]:
import numpy as np
import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import dates
matplotlib.use('TkAgg')

import seaborn as sns
import joblib
import statsmodels.api as sm
import pmdarima as pm
import plotly.graph_objects as go

from tkinter import *
import dtale as dt

# from sklearn.preprocessing import MinMaxScaler
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from sklearn.preprocessing import PowerTransformer
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller

from sklearn import metrics
from sklearn.metrics import  mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

pd.set_option('display.float_format', '{:,.6}'.format)
# pd.set_option('max_columns', 100)


In [2]:
# Standard imports 
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, save

from bokeh.models.tools import HoverTool
from bokeh.models import Legend, ColumnDataSource, Range1d
output_notebook()

Loading BokehJS ...

### Preparing the Data

In [2]:

df_enr = joblib.load("./data/GTMA Trades_df_enr")
df_sys = joblib.load("./data/GTMA Trades_df_sys")

# df_sys.head(4)

In [14]:
dt.show(df_sysM, ignore_index=True, open_browser=True)

In [3]:

#* System Trades -- Reseampling to Monthly 
df_sys_M1 = df_sys.set_index(['End Time_D&T'])[['Volume', 'Cost']].resample('M').sum()
df_sys_M2 = df_sys.set_index(['End Time_D&T'])[['Price', 'Trade Hours Duration']].resample('M').mean()
df_sys_M = df_sys_M1.join(df_sys_M2,how='left')

# df_sys_D["Cost"].head(6)

In [4]:

#* Energy Trades -- Reseampling to Monthly 
df_enr_M1 = df_enr.set_index(['End Time_D&T'])[['Volume', 'Cost']].resample('M').sum()
df_enr_M2 = df_enr.set_index(['End Time_D&T'])[['Price', 'Trade Hours Duration']].resample('M').mean()
df_enr_M = df_enr_M1.join(df_enr_M2,how='left')

df_enr_M.tail(6)

,Volume,Cost,Price,Trade Hours Duration
End Time_D&T,,,,
2021-01-31,"13,450.0",1.38888e+06,109.423,1.0
2021-02-28,-883.0,1.78777e+06,41.0662,1.00992
2021-03-31,"-19,128.0",3.79342e+05,27.3795,1.0
2021-04-30,"-9,765.0",1.81057e+06,55.6336,1.01045
2021-05-31,"-58,230.0",1.46012e+06,-18.7634,1.01138
2021-06-30,"-1,200.0","-59,438.5",49.743,1.0


In [5]:
df_sys_M.drop(labels=df_sys_M.index[-1], inplace=True)
df_enr_M.drop(labels=df_enr_M.index[-1], inplace=True)

> POWER TRANSFORM

In [6]:
joblib.dump(df_sys_M,"./data/GTMA Trades_df_sysM")
joblib.dump(df_enr_M,"./data/GTMA Trades_df_enrM")

['./data/GTMA Trades_df_enrM']

In [7]:
df_sysM = joblib.load("./data/GTMA Trades_df_sysM")
df_enrM = joblib.load("./data/GTMA Trades_df_enrM")

In [17]:

#? Testing: PwrTransformed Cost |
#! EXPERIMENT SUCCESS: Delete & Clean properly
df_sysM2 = df_sys_M

In [41]:
joblib.dump(df_sysM2,"./data/GTMA Trades_dfsysM2")


['./data/GTMA Trades_dfsysM2']

### Auxiliary Functions

In [8]:

#? Ho: It is non stationary
#? H1: It is stationary

def adfuller_test(series):
    result=adfuller(series)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("Strong evidence against Null hypothesis(Ho), Reject the null hypothesis. Data has NO unit root and IS Stationary")
    else:
        print("Weak evidence against Null hypothesis(Ho), Time series has a Unit root, indicating it IS Non-stationary")
    

In [ ]:
df_acf = acf(df_enr_M['Cost'], nlags=24)
# df_acf = acf(df_enr_D['Cost'], nlags=720)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x= np.arange(len(df_acf)),
    y= df_acf,
    name= 'ACF',
    ))

#ToDo Add 0.5 hline
# fig.add_hline(y=extr_upper, line_width=2, line_dash="dash", line_color="red",opacity=0.5)

fig.update_xaxes(rangeslider_visible=True)
fig.update_layout(
    title="Autocorrelation",
    xaxis_title="Lag",
    yaxis_title="Autocorrelation",
    #     autosize=False,
    #     width=500,
        height=500,
    )
fig.show()

In [9]:
def create_corr_plot(series, plot_pacf=False):
        corr_array = pacf(series.dropna(), nlags=(len(series)//2 -1), alpha=0.05) if plot_pacf else acf(series.dropna(),nlags=len(series), alpha=0.05)
        lower_y = corr_array[1][:,0] - corr_array[0]
        upper_y = corr_array[1][:,1] - corr_array[0]

        fig = go.Figure()
        [fig.add_scatter(x=(x,x), y=(0,corr_array[0][x]), mode='lines',line_color='#3f3f3f') for x in range(len(corr_array[0]))]
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=corr_array[0], mode='markers', marker_color='#1f77b4', marker_size=12)
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=upper_y, mode='lines', line_color='rgba(255,255,255,0)')
        fig.add_scatter(x=np.arange(len(corr_array[0])), y=lower_y, mode='lines',fillcolor='rgba(32, 146, 230,0.3)',
                        fill='tonexty', line_color='rgba(255,255,255,0)')
        fig.update_traces(showlegend=False)
        fig.update_xaxes(range=[-1,42])
        fig.update_yaxes(zerolinecolor='#000000')
        
        title='Partial Autocorrelation (PACF)' if plot_pacf else 'Autocorrelation (ACF)'
        fig.update_layout(title=title)
        fig.show()

In [39]:
def go_plotter(df_train, df_test, df_forecast, FC, LABEL, CAT):
        fig = go.Figure(layout=layout)
        fig.update_layout(title= str(CAT) + " Trades " + str(FC) + "-Month Forecast" + " -- " + LABEL)

        #? Note: Index of incoming dataframes are Datetime
        for i, t in zip([df_train, df_test, df_forecast],["Train","Test","Forecast"]):
                mode_var = 'lines+markers' if (t =="Forecast") else 'lines'
                line_dash = 'dot' if (t =="Train") else 'solid'
                fig.add_trace(go.Scatter(x=i.index, y=i["Cost_PwrTransform"].values, name=t, mode=mode_var,
                line_dash=line_dash, hovertemplate="Date = %{x}<br>Cost = %{y}"))
        # fig.for_each_trace(lambda trace: trace.update(fill='tonextx') if trace.name == "Forecast" else ())
        fig.show()

In [11]:

dict_train = {
        "Y2017_S" : (1*12) -1,
        "Y2018_M" : (1*12) + 5,
        "Y2018_S" : (2*12) -1,            #? Start of 2020
        "Y2018_M" : (2*12) + 5,            #? Midyear 2020
        "Y2019_S" : (3*12) -1,       #? Start of March
        "Y2019_M" : (3*12) + 5,
        "Y2020_S" : (4*12) -1,      #? 2020 midyear, Start of July    
        "Y2020_M" : (4*12) + 5,
        }

layout = go.Layout(
        margin=go.layout.Margin(
        l=20, #left margin
        r=15, #right margin
        b=15, #bottom margin
        t=35 ),#top margin
        xaxis_title="Date", yaxis_title="Cost (£)")

### System Trades

In [20]:
adfuller_test(df_sysM2[['Cost_PwrTransform']])

ADF Test Statistic : -3.694289544711013
p-value : 0.004194197564268018
#Lags Used : 0
Number of Observations Used : 64
Strong evidence against Null hypothesis(Ho), Reject the null hypothesis. Data has NO unit root and IS Stationary


In [13]:
adfuller_test(df_enrM["Cost"])

ADF Test Statistic : -4.533027040758231
p-value : 0.00017131492378491482
#Lags Used : 3
Number of Observations Used : 61
Strong evidence against Null hypothesis(Ho), Reject the null hypothesis. Data has NO unit root and IS Stationary


#### ADF Tests

#### ACF & PACF plots

> NEW approach 1: yeo-johnson transformation

> NEW approach 2: 2020 as Forecast year, 2019 & older are Training years

In [ ]:
df_sysM[['Cost_PwrTransform']]

In [25]:

#? EXPERIMENT with _sysM2
LABEL = 'Y2017_S'

#? Index is already DateTime. Retained Only Cost column.

# relativedelta(months=+6)
#? 59 as end of 2020. 48 start of 2020
df_train = df_sysM2[['Cost_PwrTransform']][dict_train.get(LABEL):48]
#? EXPERIMENT with 12 months Test data
df_test = df_sysM2[["Cost_PwrTransform"]][df_train.index[-1] + relativedelta(months=+1) :df_train.index[-1] + relativedelta(months=+12)]

#  (df_train.index[-1] + relativedelta(months=+FC))

FC = len(df_test.index) 

In [415]:

#? Years '18 - '20, Start &
LABEL = 'Y2017_S'

#? Index is already DateTime. Retained Only Cost column.
# .set_index("End Time_D&T", drop=True)

# relativedelta(months=+6)
# 59 as end of 2020.
df_train = df_sysM[['Cost']][dict_train.get(LABEL):47]
df_test = df_sysM[["Cost"]][df_train.index[-1] + relativedelta(months=+1) :-5]
#  (df_train.index[-1] + relativedelta(months=+FC))

FC = len(df_test.index) 

In [28]:
create_corr_plot(df_train['Cost_PwrTransform'])
# create_corr_plot(df_sys_M["Cost"][dict_train.get("Y2019_M"):-FC])

In [29]:

create_corr_plot(df_train['Cost_PwrTransform'], plot_pacf=True)
# create_corr_plot(df_sys_M["Cost"][dict_train.get(LABEL):],plot_pacf=True)


### System Trades -- Gridsearch for ARIMA 

In [34]:
arima_m = pm.auto_arima(df_train.values, start_p=0, max_p=2,
                    d=0, max_d=1, start_q=0, max_q=2,
                    seasonal=False, error_action='ignore',
                    suppress_warnings=True, trace=True, information_criterion="aicc") #maxiter=120 stepwise=False test='adf'  

Performing stepwise search to minimize aicc
 ARIMA(0,0,0)(0,0,0)[0]             : AICC=93.879, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AICC=83.865, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AICC=84.802, Time=0.03 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AICC=85.626, Time=0.08 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AICC=85.704, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AICC=inf, Time=0.14 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept   : AICC=86.239, Time=0.02 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.392 seconds


In [35]:
arima_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   37
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 -39.756
Date:                Sat, 23 Jul 2022   AIC                             83.512
Time:                        14:33:34   BIC                             86.734
Sample:                             0   HQIC                            84.648
                                 - 37                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.5328      0.161      3.305      0.001       0.217       0.849
sigma2         0.4976      0.125      3.992      0.000       0.253       0.742
===================================================================================
Ljung-Box (L1) (Q):                   0.12   Jarque-Bera (JB):                 0.37
Prob(Q):                              0.72   Prob(JB):                         0.83
Heteroskedasticity (H):               1.34   Skew:                             0.21
Prob(H) (two-sided):                  0.62   Kurtosis:                         2.75
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [36]:

#? conf_int == Confidence Interval
prediction, conf_int = arima_m.predict(n_periods=FC, return_conf_int=True)

In [414]:
cf

,0,1
0,"81,015.9",1.97594e+07
1,-2.01816e+06,2.04544e+07
2,-2.82733e+06,2.05776e+07
3,-3.20292e+06,2.06182e+07


In [40]:

#? Reference Code | Transfer to plotly.go
cf= pd.DataFrame(conf_int)

# df_forecast = pd.DataFrame(prediction, index=df_test.index.values, columns="Cost")
df_forecast = pd.DataFrame(prediction, 
                        index=df_test.index, columns=['Cost_PwrTransform'])

# FC, LABEL, CAT
go_plotter(df_train, df_test, df_forecast, FC, LABEL, "System")

In [374]:
df_forecast

,Cost
End Time_D&T,
2020-08-31,2.13345e+07
2020-09-30,2.13345e+07
2020-10-31,2.13345e+07


In [ ]:

#* Plot using GO
fig = go.Figure()
# name='lines'
fig.add_trace(go.Scatter(x=trendplot_enr_sysM["End Time_D&T"], y=prediction,
                    mode='lines', name='Trend -- System Trades')
              )
#? Add_trace for the conf_int
# fig.add_trace(go.fill?)

fig.show()

---------------------

In [ ]:
sarima_m = pm.auto_arima(TRAIN_SYS_M["Cost"], test='adf', start_p=0, max_p=4, d=0, max_d=1, start_q=0, max_q=2, seasonal=True, 
                    error_action='ignore',  
                    start_P=0, max_P=2, D=0, max_D=1, start_Q=0, max_Q=1, m=12, max_order=None,
                    suppress_warnings=True, trace=True, information_criterion="hqic", maxiter=75,
                    stepwise=False)

In [20]:
sarima_m.get_params()

{'maxiter': 50,
 'method': 'lbfgs',
 'order': (1, 1, 1),
 'out_of_sample_size': 0,
 'scoring': 'mse',
 'scoring_args': {},
 'seasonal_order': (0, 0, 0, 0),
 'start_params': None,
 'suppress_warnings': True,
 'trend': None,
 'with_intercept': False}

We consider possible orders for MA model: [0,1,2]

In [22]:
arima_m.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                   66
Model:               SARIMAX(1, 1, 1)   Log Likelihood               -1062.343
Date:                Tue, 19 Jul 2022   AIC                           2130.687
Time:                        16:03:58   BIC                           2137.210
Sample:                             0   HQIC                          2133.261
                                 - 66                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2808      0.179      1.572      0.116      -0.069       0.631
ma.L1         -0.9586      0.074    -13.011      0.000      -1.103      -0.814
sigma2       1.09e+13   1.62e-14   6.72e+26      0.000    1.09e+13    1.09e+13
===================================================================================
Ljung-Box (L1) (Q):                   0.38   Jarque-Bera (JB):               579.76
Prob(Q):                              0.54   Prob(JB):                         0.00
Heteroskedasticity (H):               0.35   Skew:                             3.05
Prob(H) (two-sided):                  0.02   Kurtosis:                        16.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number    inf. Standard errors may be unstable.
"""

In [ ]:

#? conf_int == Confidence Interval
prediction, conf_int = arima_m.predict(n_periods=TEST_SIZE, return_conf_int=True)

cf = pd.DataFrame(conf_int)



In [ ]:
prediction_series = pd.Series(prediction,index=test.index)



In [ ]:

#? Reference for train-test splitting.
train, test = data.iloc[:-TEST_SIZE], data.iloc[-TEST_SIZE:]
x_train, x_test = np.array(range(train.shape[0])), np.array(range(train.shape[0], data.shape[0]))
train.shape, x_train.shape, test.shape, x_test.shape

In [ ]:

#? Train: Historic
#? Fitting & predicting for 3 months
arima_m.fit_predict(y=,n_periods=3)

In [ ]:

#ToDo: Limit period of Training to 6 months before

## Multi-level Perceptron (MLP)

### [Bootstrap] Using Keras

### [Directly] using Scikitlearn

## References

1.) Krish Naik's YTube & github -- https://github.com/krishnaik06/ARIMA-And-Seasonal-ARIMA/blob/master/Untitled.ipynb

2.) Auto-ARIMA -- https://www.alldatascience.com/time-series/forecasting-time-series-with-auto-arima/

--------------------